Course: Computación en la nube.

Module: 3 Modelos de servicio.

Student name: Luis Felipe Castañeda Gallego.

## 1. Introduction

The purpose of this notebook is to create labels or classes for each company that won at least one contract or bid in the United States during the entire fiscal year 2023. The process of generating labels or classes is ultimately carried out using the k-means clustering method. The dataframe created will serve as raw material for applying classification models in the following notebook.

The following is a list and import of the libraries, classes, and methods used for the development of the current notebook:

In [1]:
# Standard library imports
import os
import re

# Related third-party imports
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder, StandardScaler

## 2. Load and read the data

Load the fiscal year 2023 contract data from a CSV file.

In [2]:
raw_data = pd.read_csv("fy2023_all_contracts.csv")

Adjust the pandas display settings to allow up to 300 columns to be shown when displaying DataFrame information.

In [3]:
pd.set_option('display.max_info_columns', 300)

Display the structure and column-wise non-null count of the raw_data DataFrame.

In [4]:
raw_data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6669114 entries, 0 to 6669113
Data columns (total 27 columns):
 #   Column                                     Non-Null Count    Dtype  
---  ------                                     --------------    -----  
 0   federal_action_obligation                  6669114 non-null  float64
 1   total_dollars_obligated                    6669114 non-null  float64
 2   current_total_value_of_award               6431557 non-null  float64
 3   potential_total_value_of_award             6669114 non-null  float64
 4   period_of_performance_start_date           6669101 non-null  object 
 5   period_of_performance_current_end_date     6431557 non-null  object 
 6   period_of_performance_potential_end_date   6431557 non-null  object 
 7   awarding_agency_name                       6669114 non-null  object 
 8   funding_agency_name                        6669114 non-null  object 
 9   recipient_uei                              6669113 non-null  object 

Compute and display the frequency of the unique values of some columns of interest.

In [5]:
raw_data['awarding_agency_name'].value_counts()

awarding_agency_name
Department of Defense                    4396185
General Services Administration          1555247
Department of Veterans Affairs            124811
Department of Justice                      99095
Department of State                        74094
                                          ...   
Library of Congress                           33
United States Chemical Safety Board           19
National Endowment for the Humanities         12
Federal Housing Finance Agency                 7
Administrative Conference of the U.S.          4
Name: count, Length: 69, dtype: int64

In [6]:
raw_data['funding_agency_name'].value_counts()

funding_agency_name
Department of Defense                      4388917
General Services Administration            1541244
Department of Veterans Affairs              125167
Department of Justice                        98254
Department of Health and Human Services      73079
                                            ...   
Administrative Conference of the U.S.            5
U.S. Postal Service                              5
American Battle Monuments Commission             5
Intelligence Community Staff                     2
U.S. Tax Court                                   1
Name: count, Length: 92, dtype: int64

In [7]:
raw_data['recipient_uei'].value_counts()

recipient_uei
KABZK8W6PQT3    464585
MA1VZ6667CB1    338655
ZJEUBM5FYLQ2    282944
E8QNBC287DY4    276023
YLE5AAYNVPK6    168739
                 ...  
K26GHT8N8LD9         1
WNNMPKGA5KH6         1
DUKZQ3KNZ2N6         1
LZQMTMARC8Q3         1
WSN5GV1EBE61         1
Name: count, Length: 109396, dtype: int64

In [8]:
raw_data['recipient_name'].value_counts()

recipient_name
AMERISOURCEBERGEN DRUG CORP               464585
ASRC FEDERAL FACILITIES LOGISTICS, LLC    338656
CARDINAL HEALTH 200, LLC                  282944
NATIONAL INDUSTRIES FOR THE BLIND         276023
NOBLE SUPPLY & LOGISTICS, LLC             168740
                                           ...  
STOLZ TELECOM LLC                              1
FAIRFIELD INDUSTRIES INCORPORATED              1
NALCO CONSTRUCTION COMPANY                     1
MILNER TECHNOLOGIES INC                        1
NAFTALY CHRISTINA SITOMPUL                     1
Name: count, Length: 106037, dtype: int64

In [9]:
raw_data['recipient_country_name'].value_counts()

recipient_country_name
UNITED STATES        6524750
GERMANY                21619
JAPAN                  16871
SWITZERLAND            16842
UNITED KINGDOM         12464
                      ...   
SYRIA                      2
CURACAO                    2
SAINT LUCIA                1
LIBYA                      1
EQUATORIAL GUINEA          1
Name: count, Length: 179, dtype: int64

In [10]:
raw_data['recipient_state_code'].value_counts()

recipient_state_code
VA         838999
IL         719288
PA         655484
NJ         618665
CA         412055
            ...  
PW             35
MH              8
AP              5
FM              4
GAUTENG         2
Name: count, Length: 61, dtype: int64

In [11]:
raw_data['recipient_state_name'].value_counts()

recipient_state_name
VIRGINIA         838999
ILLINOIS         719288
PENNSYLVANIA     655484
NEW JERSEY       618665
CALIFORNIA       412055
                  ...  
TH-BANGKOK            1
ILIDZA                1
UTTAR PRADESH         1
YAOUNDE               1
PAVIA                 1
Name: count, Length: 713, dtype: int64

In [12]:
raw_data['primary_place_of_performance_country_code'].value_counts()

primary_place_of_performance_country_code
USA    6246168
CHE      29621
JPN      19612
DEU      12105
KWT      11285
        ...   
KIR          1
HMD          1
ESH          1
AND          1
BLM          1
Name: count, Length: 216, dtype: int64

In [13]:
raw_data['primary_place_of_performance_country_name'].value_counts()

primary_place_of_performance_country_name
UNITED STATES                        6246168
SWITZERLAND                            29621
JAPAN                                  19612
GERMANY                                12105
KUWAIT                                 11285
                                      ...   
KIRIBATI                                   1
HEARD ISLAND AND MCDONALD ISLANDS          1
WESTERN SAHARA                             1
ANDORRA                                    1
SAINT BARTHELEMY                           1
Name: count, Length: 216, dtype: int64

In [14]:
raw_data['primary_place_of_performance_state_code'].value_counts()

primary_place_of_performance_state_code
IL    707915
PA    647958
VA    612660
NJ    558264
CA    385063
NY    366495
TX    285922
FL    218449
MA    217333
GA    190149
NC    184467
MI    182340
MO    148150
MD    139931
OH    119943
WI    111080
WA    109541
SC     93825
DC     92148
AL     66802
CO     60276
CT     56175
MS     55475
AZ     46137
MN     45605
IN     44703
AR     42615
KS     41319
HI     40365
LA     38516
OR     35664
OK     29724
TN     27214
KY     21998
NE     20660
IA     20576
NV     18696
NM     17898
ID     17831
UT     16450
AK     14680
ME     12966
MT     12679
WV     10679
GU      8074
ND      7796
NH      6848
SD      6672
RI      6367
DE      6171
PR      5957
VT      5617
WY      4237
VI       489
MP       187
MH       158
PW       141
AS        75
FM        43
Name: count, dtype: int64

In [15]:
raw_data['primary_place_of_performance_state_name'].value_counts()

primary_place_of_performance_state_name
ILLINOIS                          707915
PENNSYLVANIA                      647958
VIRGINIA                          612660
NEW JERSEY                        558264
CALIFORNIA                        385063
NEW YORK                          366495
TEXAS                             285922
FLORIDA                           218449
MASSACHUSETTS                     217333
GEORGIA                           190149
NORTH CAROLINA                    184467
MICHIGAN                          182340
MISSOURI                          148150
MARYLAND                          139931
OHIO                              119943
WISCONSIN                         111080
WASHINGTON                        109541
SOUTH CAROLINA                     93825
DISTRICT OF COLUMBIA               92148
ALABAMA                            66802
COLORADO                           60276
CONNECTICUT                        56175
MISSISSIPPI                        55475
ARIZONA          

In [16]:
raw_data['award_or_idv_flag'].value_counts()

award_or_idv_flag
AWARD    6431557
IDV       237557
Name: count, dtype: int64

In [17]:
raw_data['award_type'].value_counts()

award_type
DELIVERY ORDER         4524241
BPA CALL                943859
PURCHASE ORDER          801816
DEFINITIVE CONTRACT     161641
Name: count, dtype: int64

In [18]:
raw_data['product_or_service_code'].value_counts()

product_or_service_code
8915    877393
6505    537629
6515    502099
5120    462739
7510    407031
         ...  
1070         1
E1FA         1
E1LB         1
AG94         1
H237         1
Name: count, Length: 2544, dtype: int64

In [19]:
raw_data['product_or_service_code_description'].value_counts()

product_or_service_code_description
FRUITS AND VEGETABLES                                                                           877393
DRUGS AND BIOLOGICALS                                                                           537629
MEDICAL AND SURGICAL INSTRUMENTS, EQUIPMENT, AND SUPPLIES                                       502099
HAND TOOLS, NONEDGED, NONPOWERED                                                                462739
OFFICE SUPPLIES                                                                                 407031
                                                                                                 ...  
RDTE/SP+TERRESTRIAL APP-ENG/MANUF                                                                    1
INSPECTION- LIGHTING FIXTURES AND LAMPS                                                              1
NATURAL RESOURCES AND ENVIRONMENT R&D SERVICES; WATER RESOURCES; R&D ADMINISTRATIVE EXPENSES         1
R&D- ECONOMIC GROWTH: MANUFACTURING T

In [20]:
raw_data['type_of_contract_pricing'].value_counts()

type_of_contract_pricing
FIRM FIXED PRICE                                                                               4733385
FIXED PRICE WITH ECONOMIC PRICE ADJUSTMENT                                                     1679276
COST PLUS FIXED FEE                                                                              97052
TIME AND MATERIALS                                                                               48029
LABOR HOURS                                                                                      36310
ORDER DEPENDENT (IDV ALLOWS PRICING ARRANGEMENT TO BE DETERMINED SEPARATELY FOR EACH ORDER)      30584
COST NO FEE                                                                                      13014
FIXED PRICE REDETERMINATION                                                                       8835
COST PLUS AWARD FEE                                                                               8221
COST PLUS INCENTIVE FEE                         

In [21]:
raw_data['naics_code'].value_counts()

naics_code
423450.0    719939
311812.0    484836
311991.0    452378
424210.0    353375
324110.0    232449
             ...  
424810.0         1
454210.0         1
111920.0         1
332116.0         1
453930.0         1
Name: count, Length: 1164, dtype: int64

In [22]:
raw_data['naics_description'].value_counts()

naics_description
MEDICAL, DENTAL, AND HOSPITAL EQUIPMENT AND SUPPLIES MERCHANT WHOLESALERS    719939
COMMERCIAL BAKERIES                                                          484836
PERISHABLE PREPARED FOOD MANUFACTURING                                       452378
DRUGS AND DRUGGISTS' SUNDRIES MERCHANT WHOLESALERS                           353375
PETROLEUM REFINERIES                                                         232449
                                                                              ...  
VENDING MACHINE OPERATORS                                                         1
COTTON FARMING                                                                    1
CATTLE FEEDLOTS                                                                   1
MONETARY AUTHORITIES - CENTRAL BANK                                               1
FRUIT AND VEGETABLE MARKETS                                                       1
Name: count, Length: 1135, dtype: int64

In [23]:
raw_data['extent_competed'].value_counts()

extent_competed
FULL AND OPEN COMPETITION                               4160765
COMPETED UNDER SAP                                       913159
FULL AND OPEN COMPETITION AFTER EXCLUSION OF SOURCES     661151
NOT AVAILABLE FOR COMPETITION                            395058
NOT COMPETED                                             310086
NOT COMPETED UNDER SAP                                   208059
COMPETITIVE DELIVERY ORDER                                  106
FOLLOW ON TO COMPETED ACTION                                 50
NON-COMPETITIVE DELIVERY ORDER                               38
Name: count, dtype: int64

In [24]:
raw_data['c8a_program_participant'].value_counts()

c8a_program_participant
f    6518767
t     150347
Name: count, dtype: int64

## 3. Preprocessing the data

Select the columns of interest.

In [25]:
columns = [
    "federal_action_obligation",
    "awarding_agency_name",
    "funding_agency_name",
    "recipient_uei",
    "recipient_country_name",
    "recipient_state_code",
    "primary_place_of_performance_country_code",
    "primary_place_of_performance_state_code",
    "award_or_idv_flag",
    "award_type",
    "type_of_contract_pricing",
    "naics_code",
    "extent_competed",
    "c8a_program_participant",
    ]

Create a dataframe just with the columns of interest.

In [26]:
df_1 = raw_data[columns]

Display the first five rows of df_1.

In [27]:
df_1.head()

,federal_action_obligation,awarding_agency_name,funding_agency_name,recipient_uei,recipient_country_name,recipient_state_code,primary_place_of_performance_country_code,primary_place_of_performance_state_code,award_or_idv_flag,award_type,type_of_contract_pricing,naics_code,extent_competed,c8a_program_participant
0,46203.00,Department of Defense,Department of Defense,DJRUN4KK1HK3,UNITED STATES,DC,USA,DC,AWARD,DELIVERY ORDER,FIRM FIXED PRICE,541512.0,FULL AND OPEN COMPETITION,f
1,1302.72,Department of Defense,Department of Defense,KABZK8W6PQT3,UNITED STATES,PA,USA,PA,AWARD,DELIVERY ORDER,FIRM FIXED PRICE,325411.0,FULL AND OPEN COMPETITION,f
2,3721646.00,Department of Defense,Department of Defense,JCBMLGPE6Z71,UNITED STATES,VA,USA,MD,AWARD,DELIVERY ORDER,COST PLUS FIXED FEE,541330.0,FULL AND OPEN COMPETITION,f
3,0.00,Department of Veterans Affairs,Department of Veterans Affairs,C1G1CAQGHZE8,UNITED STATES,FL,NaN,NaN,IDV,NaN,FIRM FIXED PRICE,532490.0,COMPETED UNDER SAP,f
4,-400.74,General Services Administration,General Services Administration,MF46EHTMKJJ5,UNITED STATES,MI,USA,MI,AWARD,DELIVERY ORDER,FIXED PRICE WITH ECONOMIC PRICE ADJUSTMENT,423710.0,FULL AND OPEN COMPETITION,f


Output detailed information about df_1.

In [28]:
df_1.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6669114 entries, 0 to 6669113
Data columns (total 14 columns):
 #   Column                                     Non-Null Count    Dtype  
---  ------                                     --------------    -----  
 0   federal_action_obligation                  6669114 non-null  float64
 1   awarding_agency_name                       6669114 non-null  object 
 2   funding_agency_name                        6669114 non-null  object 
 3   recipient_uei                              6669113 non-null  object 
 4   recipient_country_name                     6668859 non-null  object 
 5   recipient_state_code                       6524737 non-null  object 
 6   primary_place_of_performance_country_code  6431557 non-null  object 
 7   primary_place_of_performance_state_code    6246168 non-null  object 
 8   award_or_idv_flag                          6669114 non-null  object 
 9   award_type                                 6431557 non-null  object 

Duplicate df_1 to create an independent copy.

In [29]:
df_1 = df_1.copy()

### 3.1. Data cleaning

Refine df_1 by replacing empty strings in the 'recipient_uei' column with NA (Not Available) and then removing any rows where 'recipient_uei' is NA.

In [30]:
df_1 = df_1[df_1['recipient_uei'].replace('', pd.NA).notna()]

Filter df_1 to exclude rows where 'federal_action_obligation' is either negative or not available.

In [31]:
df_1 = df_1[df_1['federal_action_obligation'].apply(lambda x: x >= 0 if pd.notna(x) else False)]

Narrow down df_1 to include only those records where the primary place of performance country code is 'USA'.

In [32]:
df_1 = df_1[df_1['primary_place_of_performance_country_code'] == 'USA']

Create a function named clean_naics_code which is used to process NAICS (North American Industry Classification System) codes. The function ensures that any non-null NAICS codes are converted to integers and then back to strings, to standardize the format and to remove any floating-point artifacts or formatting inconsistencies.

In [33]:
def clean_naics_code(naics_code):
    if pd.notnull(naics_code):
        naics_code = str(int(naics_code))
    return naics_code

Apply naics code cleaning function.

In [34]:
df_1.loc[:, 'naics_code'] = df_1['naics_code'].apply(clean_naics_code).astype(str)

C:\Users\lufec\AppData\Local\Temp\ipykernel_2576\3258493098.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['541512' '325411' '541330' ... '334111' '334111' '541611']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_1.loc[:, 'naics_code'] = df_1['naics_code'].apply(clean_naics_code).astype(str)


Create a function called clean_text_column, to clean text data within a DataFrame. The function standardizes text entries: it removes all punctuation, strips leading and trailing spaces, and converts the text to lowercase.

In [35]:
def clean_text_column(text):
    if pd.notnull(text):
        
        text = re.sub(r'[^\w\s]', '', text)
        text = text.strip().lower()
    return text

Retrieve a list of all columns in df_1 that are of the object data type.

In [36]:
object_columns = df_1.select_dtypes(include='object').columns.tolist()

Apply the clean_text_column function to each column identified as containing object-type data in df_1.

In [37]:
for col in object_columns:
    df_1[col] = df_1[col].apply(clean_text_column)

Display the first five rows of df_1.

In [38]:
df_1.head()

,federal_action_obligation,awarding_agency_name,funding_agency_name,recipient_uei,recipient_country_name,recipient_state_code,primary_place_of_performance_country_code,primary_place_of_performance_state_code,award_or_idv_flag,award_type,type_of_contract_pricing,naics_code,extent_competed,c8a_program_participant
0,46203.00,department of defense,department of defense,djrun4kk1hk3,united states,dc,usa,dc,award,delivery order,firm fixed price,541512,full and open competition,f
1,1302.72,department of defense,department of defense,kabzk8w6pqt3,united states,pa,usa,pa,award,delivery order,firm fixed price,325411,full and open competition,f
2,3721646.00,department of defense,department of defense,jcbmlgpe6z71,united states,va,usa,md,award,delivery order,cost plus fixed fee,541330,full and open competition,f
6,180877.78,department of state,department of state,c47bna8gm833,united states,va,usa,va,award,bpa call,labor hours,541519,full and open competition after exclusion of s...,f
7,0.00,department of state,department of state,c47bna8gm833,united states,va,usa,va,award,bpa call,labor hours,541519,full and open competition after exclusion of s...,f


Output a column-wise count of non-null values of df_1.

In [39]:
df_1.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 5982560 entries, 0 to 6669113
Data columns (total 14 columns):
 #   Column                                     Non-Null Count    Dtype  
---  ------                                     --------------    -----  
 0   federal_action_obligation                  5982560 non-null  float64
 1   awarding_agency_name                       5982560 non-null  object 
 2   funding_agency_name                        5982560 non-null  object 
 3   recipient_uei                              5982560 non-null  object 
 4   recipient_country_name                     5982396 non-null  object 
 5   recipient_state_code                       5979178 non-null  object 
 6   primary_place_of_performance_country_code  5982560 non-null  object 
 7   primary_place_of_performance_state_code    5982560 non-null  object 
 8   award_or_idv_flag                          5982560 non-null  object 
 9   award_type                                 5982560 non-null  object 
 10 

Duplicate the cleaned DataFrame df_1 to df_2.

In [40]:
df_2 = df_1.copy()

Create the create_n_digit_naics function, to modify NAICS codes within a DataFrame to their first three digits. Truncating the NAICS codes can be useful in analyses that require less granularity and more general industry insights.

In [41]:
def create_n_digit_naics(df):
    n = 3
    
    def extract_n_digits(code):
        if isinstance(code, str) and code.isdigit():
            return code[:n]
        else:
            return 'unknown'
    
    df['naics_code'] = df['naics_code'].apply(extract_n_digits)
    return df

Apply the create_n_digit_naics function to truncate the NAICS codes in df_2 to the first three digits.

In [42]:
df_2 = create_n_digit_naics(df_2)

Select and display a random sample of five entries from df_2.

In [43]:
df_2.sample(n=5)

,federal_action_obligation,awarding_agency_name,funding_agency_name,recipient_uei,recipient_country_name,recipient_state_code,primary_place_of_performance_country_code,primary_place_of_performance_state_code,award_or_idv_flag,award_type,type_of_contract_pricing,naics_code,extent_competed,c8a_program_participant
6284327,295.36,department of defense,department of defense,he7njqgp4ed6,united states,pa,usa,pa,award,delivery order,fixed price with economic price adjustment,423,full and open competition,f
2201770,9623.10,department of defense,department of defense,tvjcf48t79l9,united states,nj,usa,nj,award,delivery order,fixed price with economic price adjustment,423,not competed,f
4921734,191.62,general services administration,general services administration,swfsjms9ehb2,united states,nc,usa,nc,award,bpa call,fixed price with economic price adjustment,339,full and open competition,f
2616094,17134.78,department of defense,department of defense,t6jhke96ea19,united states,oh,usa,oh,award,delivery order,fixed price redetermination,334,not competed,f
4303943,16.42,general services administration,general services administration,sjl9q452qbl5,united states,wi,usa,wi,award,bpa call,firm fixed price,332,full and open competition,f


Output a column-wise count of non-null values of df_2.

In [44]:
df_2.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 5982560 entries, 0 to 6669113
Data columns (total 14 columns):
 #   Column                                     Non-Null Count    Dtype  
---  ------                                     --------------    -----  
 0   federal_action_obligation                  5982560 non-null  float64
 1   awarding_agency_name                       5982560 non-null  object 
 2   funding_agency_name                        5982560 non-null  object 
 3   recipient_uei                              5982560 non-null  object 
 4   recipient_country_name                     5982396 non-null  object 
 5   recipient_state_code                       5979178 non-null  object 
 6   primary_place_of_performance_country_code  5982560 non-null  object 
 7   primary_place_of_performance_state_code    5982560 non-null  object 
 8   award_or_idv_flag                          5982560 non-null  object 
 9   award_type                                 5982560 non-null  object 
 10 

### 3.2. Aggregation by recipient

Perform an aggregation on df_2 grouped by the unique entity identifier (recipient_uei) of each recipient. The aggregation summs the federal obligations and extracts the most frequent (mode) values for various categorical attributes. This method provides a summarized view of each recipient.

In [45]:
df_3 = df_2.groupby('recipient_uei').agg({
    
    'federal_action_obligation': "sum",
    'recipient_country_name': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown',
    'recipient_state_code': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown',
    'primary_place_of_performance_state_code': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown',
    'award_or_idv_flag': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown',
    'award_type': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown',
    'type_of_contract_pricing': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown',
    'naics_code': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown',
    'extent_competed': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown',
    'c8a_program_participant': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown',    
}).reset_index()

Add a new column to df_3, which counts the total number of contracts associated with each recipient (recipient_uei) in df_2.

In [46]:
df_3['number_of_contracts'] = df_2.groupby('recipient_uei').size().reset_index(name='number_of_contracts')['number_of_contracts']

Display the first five rows of df_3.

In [47]:
df_3.head()

,recipient_uei,federal_action_obligation,recipient_country_name,recipient_state_code,primary_place_of_performance_state_code,award_or_idv_flag,award_type,type_of_contract_pricing,naics_code,extent_competed,c8a_program_participant,number_of_contracts
0,c111jjbms328,45935.00,united states,ca,ca,award,purchase order,firm fixed price,337,not competed under sap,f,1
1,c112zns5hmr4,4463.00,united states,dc,dc,award,bpa call,firm fixed price,541,competed under sap,f,4
2,c113jmw3wgs7,171001.76,united states,tx,tx,award,purchase order,firm fixed price,541,competed under sap,f,3
3,c114qv5r7yx8,7515586.92,united states,in,in,award,delivery order,firm fixed price,311,full and open competition,f,11
4,c116e6t5mgd6,1702950.88,united states,nh,co,award,delivery order,firm fixed price,334,full and open competition,f,3


Check the number of contracts for any (of the first five) recipients in df_3.

In [48]:
count = (df_2['recipient_uei'] == 'c114qv5r7yx8').sum()
print("Number of rows in 'recipient_uei':", count)

Number of rows in 'recipient_uei': 11


Show the structure, data types, and column information for df_3.

In [49]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86145 entries, 0 to 86144
Data columns (total 12 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   recipient_uei                            86145 non-null  object 
 1   federal_action_obligation                86145 non-null  float64
 2   recipient_country_name                   86145 non-null  object 
 3   recipient_state_code                     86145 non-null  object 
 4   primary_place_of_performance_state_code  86145 non-null  object 
 5   award_or_idv_flag                        86145 non-null  object 
 6   award_type                               86145 non-null  object 
 7   type_of_contract_pricing                 86145 non-null  object 
 8   naics_code                               86145 non-null  object 
 9   extent_competed                          86145 non-null  object 
 10  c8a_program_participant                  86145

Compute, show and check the frequency of the unique values of the columns in df_3.

In [50]:
df_3["recipient_country_name"].unique()

array(['united states', 'canada', 'australia', 'united kingdom',
       'ireland', 'switzerland', 'united arab emirates', 'germany',
       'kenya', 'south africa', 'bermuda', 'finland', 'kuwait',
       'bulgaria', 'turkey', 'chile', 'spain', 'iraq', 'austria', 'italy',
       'Unknown', 'serbia', 'ukraine', 'kyrgyzstan', 'czechia',
       'portugal', 'israel', 'panama', 'brazil', 'india', 'france',
       'singapore', 'romania', 'peru', 'norway', 'congo brazzaville',
       'netherlands', 'lebanon', 'croatia', 'indonesia', 'new zealand',
       'belgium', 'korea south', 'kazakhstan', 'malta', 'luxembourg',
       'denmark', 'japan', 'poland', 'pakistan', 'burma', 'philippines',
       'malawi', 'hong kong', 'russia', 'sweden', 'lithuania', 'rwanda',
       'cameroon', 'ecuador', 'estonia', 'colombia', 'saint lucia',
       'greece', 'haiti', 'madagascar', 'saudi arabia', 'honduras',
       'dominican republic', 'mexico', 'morocco', 'uruguay', 'mozambique',
       'jordan', 'afghanist

In [51]:
df_3["recipient_state_code"].unique()

array(['ca', 'dc', 'tx', 'in', 'nh', 'ny', 'ga', 'fl', 'il', 'md', 'nj',
       'va', 'ks', 'mt', 'co', 'ma', 'az', 'Unknown', 'wa', 'mo', 'nc',
       'or', 'ak', 'hi', 'ne', 'oh', 'ok', 'nv', 'sc', 'ct', 'ms', 'la',
       'pa', 'nm', 'id', 'sd', 'mi', 'tn', 'ut', 'mn', 'al', 'pr', 'wv',
       'nd', 'ia', 'gu', 'me', 'ky', 'wi', 'ar', 'ri', 'de', 'wy', 'vt',
       'mp', 'vi', 'fm', 'as', 'pw', 'mh'], dtype=object)

In [52]:
df_3["primary_place_of_performance_state_code"].unique()

array(['ca', 'dc', 'tx', 'in', 'co', 'ny', 'ga', 'fl', 'il', 'md', 'nj',
       'ks', 'mt', 'va', 'ma', 'az', 'pa', 'oh', 'ok', 'mo', 'nc', 'or',
       'al', 'gu', 'ne', 'nv', 'wa', 'ky', 'ct', 'ms', 'ak', 'la', 'nm',
       'sd', 'mi', 'tn', 'ut', 'mn', 'wv', 'nd', 'wi', 'pr', 'id', 'ia',
       'sc', 'ar', 'me', 'hi', 'ri', 'nh', 'de', 'wy', 'vt', 'mp', 'vi',
       'mh', 'fm', 'as', 'pw'], dtype=object)

In [53]:
df_3["award_or_idv_flag"].unique()

array(['award'], dtype=object)

In [54]:
df_3["award_type"].unique()

array(['purchase order', 'bpa call', 'delivery order',
       'definitive contract'], dtype=object)

In [55]:
df_3["type_of_contract_pricing"].unique()

array(['firm fixed price', 'cost plus fixed fee',
       'fixed price with economic price adjustment', 'cost no fee',
       'time and materials',
       'other applies to awards where none of the above apply',
       'labor hours', 'cost plus award fee', 'cost plus incentive fee',
       'fixed price incentive', 'fixed price award fee',
       'fixed price level of effort', 'cost sharing', 'Unknown',
       'fixed price redetermination',
       'combination applies to awards where two or more of the above apply'],
      dtype=object)

In [56]:
df_3["naics_code"].unique()

array(['337', '541', '311', '334', '621', '332', '423', '624', '336',
       '561', '111', '325', '517', '623', '327', '511', '611', '339',
       '326', '238', '518', '513', '488', '811', '721', '922', '622',
       '531', '812', '532', '237', '335', '236', '115', '221', '493',
       '324', '562', '483', 'unknown', '519', '813', '921', '212', '484',
       '711', '444', '112', '333', '323', '424', '485', '443', '312',
       '722', '923', '524', '321', '492', '512', '459', '928', '331',
       '449', '313', '713', '322', '211', '441', '113', '523', '315',
       '522', '213', '481', '926', '114', '712', '491', '314', '515',
       '316', '924', '453', '516', '425', '814', '551', '448', '487',
       '486', '454', '457', '458', '927', '445', '525', '456', '446',
       '234', '451', '442', '455', '482', '533'], dtype=object)

In [57]:
df_3["extent_competed"].unique()

array(['not competed under sap', 'competed under sap',
       'full and open competition', 'not competed',
       'full and open competition after exclusion of sources',
       'not available for competition', 'competitive delivery order',
       'Unknown', 'noncompetitive delivery order',
       'follow on to competed action'], dtype=object)

In [58]:
df_3["c8a_program_participant"].unique()

array(['f', 't'], dtype=object)

Update the names of several columns in df_3 to make them more concise.

In [59]:
df_3 = df_3.rename(columns={
    'federal_action_obligation': 'federal_action_sum',
    'recipient_country_name': 'recipient_country',
    'recipient_state_code': 'recipient_state',
    'primary_place_of_performance_state_code': 'state_of_performance',
    'naics_code': 'naics',
    'c8a_program_participant': 'c8a_program',
})

Check the new column names.

In [60]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86145 entries, 0 to 86144
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   recipient_uei             86145 non-null  object 
 1   federal_action_sum        86145 non-null  float64
 2   recipient_country         86145 non-null  object 
 3   recipient_state           86145 non-null  object 
 4   state_of_performance      86145 non-null  object 
 5   award_or_idv_flag         86145 non-null  object 
 6   award_type                86145 non-null  object 
 7   type_of_contract_pricing  86145 non-null  object 
 8   naics                     86145 non-null  object 
 9   extent_competed           86145 non-null  object 
 10  c8a_program               86145 non-null  object 
 11  number_of_contracts       86145 non-null  int64  
dtypes: float64(1), int64(1), object(10)
memory usage: 7.9+ MB


### 3.3. Encoding numerical and categorical features

Convert several descriptive columns in df_3 to the 'category' data type.

In [61]:
columns_to_convert = [
    'recipient_country',
    'recipient_state',
    'state_of_performance',
    'award_type',
    'type_of_contract_pricing',
    'naics',
    'extent_competed',
    'c8a_program',    
    ]  
for col in columns_to_convert:
    df_3[col] = df_3[col].astype('category')

Check the columns type transformation.

In [62]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86145 entries, 0 to 86144
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   recipient_uei             86145 non-null  object  
 1   federal_action_sum        86145 non-null  float64 
 2   recipient_country         86145 non-null  category
 3   recipient_state           86145 non-null  category
 4   state_of_performance      86145 non-null  category
 5   award_or_idv_flag         86145 non-null  object  
 6   award_type                86145 non-null  category
 7   type_of_contract_pricing  86145 non-null  category
 8   naics                     86145 non-null  category
 9   extent_competed           86145 non-null  category
 10  c8a_program               86145 non-null  category
 11  number_of_contracts       86145 non-null  int64   
dtypes: category(8), float64(1), int64(1), object(2)
memory usage: 3.3+ MB


Create a copy of the aggregated and processed DataFrame from df_3 to df_4.

In [63]:
df_4 = df_3.copy()

Reorganize and Select the most relevant columns in df_4.

In [64]:
df_4 = df_4[
    [
        "federal_action_sum",
        "recipient_country",
        "recipient_state",
        "state_of_performance",
        "award_type",
        "type_of_contract_pricing",
        "naics",
        "extent_competed",
        "c8a_program",        
        "number_of_contracts",
        ]]

Check for duplicated rows (may be for a posterior analysis or training a model).

In [65]:
duplicates = df_4.duplicated(keep=False)  
num_duplicates = duplicates.sum()
num_duplicates

2026

Display the updated structure of df_4.

In [66]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86145 entries, 0 to 86144
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   federal_action_sum        86145 non-null  float64 
 1   recipient_country         86145 non-null  category
 2   recipient_state           86145 non-null  category
 3   state_of_performance      86145 non-null  category
 4   award_type                86145 non-null  category
 5   type_of_contract_pricing  86145 non-null  category
 6   naics                     86145 non-null  category
 7   extent_competed           86145 non-null  category
 8   c8a_program               86145 non-null  category
 9   number_of_contracts       86145 non-null  int64   
dtypes: category(8), float64(1), int64(1)
memory usage: 2.0 MB


Compute, display and recheck the frequency of the unique values of the relevant columns.

In [67]:
df_4["recipient_country"].unique()

['united states', 'canada', 'australia', 'united kingdom', 'ireland', ..., 'senegal', 'latvia', 'maldives', 'uganda', 'gibraltar']
Length: 87
Categories (87, object): ['Unknown', 'afghanistan', 'australia', 'austria', ..., 'united arab emirates', 'united kingdom', 'united states', 'uruguay']

In [68]:
df_4["recipient_state"].unique()

['ca', 'dc', 'tx', 'in', 'nh', ..., 'vi', 'fm', 'as', 'pw', 'mh']
Length: 60
Categories (60, object): ['Unknown', 'ak', 'al', 'ar', ..., 'wa', 'wi', 'wv', 'wy']

In [69]:
df_4["state_of_performance"].unique()

['ca', 'dc', 'tx', 'in', 'co', ..., 'vi', 'mh', 'fm', 'as', 'pw']
Length: 59
Categories (59, object): ['ak', 'al', 'ar', 'as', ..., 'wa', 'wi', 'wv', 'wy']

In [70]:
df_4["award_type"].unique()

['purchase order', 'bpa call', 'delivery order', 'definitive contract']
Categories (4, object): ['bpa call', 'definitive contract', 'delivery order', 'purchase order']

In [71]:
df_4["type_of_contract_pricing"].unique()

['firm fixed price', 'cost plus fixed fee', 'fixed price with economic price adjustment', 'cost no fee', 'time and materials', ..., 'fixed price level of effort', 'cost sharing', 'Unknown', 'fixed price redetermination', 'combination applies to awards where two or mo...]
Length: 16
Categories (16, object): ['Unknown', 'combination applies to awards where two or mo..., 'cost no fee', 'cost plus award fee', ..., 'fixed price with economic price adjustment', 'labor hours', 'other applies to awards where none of the abo..., 'time and materials']

In [72]:
df_4["naics"].unique()

['337', '541', '311', '334', '621', ..., '451', '442', '455', '482', '533']
Length: 105
Categories (105, object): ['111', '112', '113', '114', ..., '926', '927', '928', 'unknown']

In [73]:
df_4["extent_competed"].unique()

['not competed under sap', 'competed under sap', 'full and open competition', 'not competed', 'full and open competition after exclusion of ..., 'not available for competition', 'competitive delivery order', 'Unknown', 'noncompetitive delivery order', 'follow on to competed action']
Categories (10, object): ['Unknown', 'competed under sap', 'competitive delivery order', 'follow on to competed action', ..., 'noncompetitive delivery order', 'not available for competition', 'not competed', 'not competed under sap']

In [74]:
df_4["c8a_program"].unique()

['f', 't']
Categories (2, object): ['f', 't']

Identify and extract all columns of the 'category' data type from df_4 into a new DataFrame df_4_cat.

In [75]:
df_4_cat_cols = df_4.select_dtypes(include=['category']).columns
df_4_cat = df_4[df_4_cat_cols]

Identify and extract all columns of the 'numerical' data type from df_4 into a new DataFrame df_4_num.

In [76]:
df_4_num_cols = df_4.select_dtypes(include=['float64', 'int64']).columns
df_4_num = df_4[df_4_num_cols]

Encode categorical columns with One-Hot encoding.

In [77]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
df_4_cat_encoded = encoder.fit_transform(df_4_cat)
df_4_cat_encoded = pd.DataFrame(
    df_4_cat_encoded,
    columns=encoder.get_feature_names_out(),
    )

Scale the numerical columns in df_4 applying standard scaling  using the StandardScaler.

In [78]:
scaler = StandardScaler()
df_4_num_scaled = scaler.fit_transform(df_4_num)
df_4_num_scaled = pd.DataFrame(df_4_num_scaled, columns=df_4_num_cols)

Combine processed data merging he one-hot encoded categorical data and the standardized numerical data into a single DataFrame called df_4_processed.

In [79]:
df_4_processed = pd.concat([df_4_cat_encoded, df_4_num_scaled], axis=1)

Check the dimensions of the processed DataFrame.

In [80]:
df_4_processed.shape

(86145, 345)

## 4. K-means clustering

Determine the optimal cluster count using the elbow method for k-means clustering on the df_4_processed DataFrame.

In [81]:
os.environ['OMP_NUM_THREADS'] = '1'
elbow_inertias = []
k_range = range(1, 16)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_4_processed.values)
    elbow_inertias.append(kmeans.inertia_)

Visualize the elbow method results.

In [82]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=np.array(k_range),
        y=elbow_inertias,
        mode='markers+lines',
        marker=dict(symbol='circle', size=6, color='blue'),
        line=dict(color='blue', width=2),
        )
        )

fig.update_layout(
    width=500,
    height=500,
    xaxis_dtick=1,
    xaxis_title='Number of clusters',
    xaxis_titlefont_size=20,
    xaxis_tickfont_size=15,
    yaxis_dtick=50000,
    yaxis_title='Inertia',
    yaxis_titlefont_size=20,
    yaxis_tickfont_size=15,
    margin=dict(t=15, b=15, l=15, r=15),
    ) 

fig.show()

The plot shows a drop in inertia as the number of clusters increases from 1 to around 5 or 6. There is a change in the rate of decrease in inertia around 5 or 6 clusters, after this point, the curve begins to flatten, indicating that additional clusters result in diminishing improvements to the model's inertia. They will be considered 5 and 6 clusters. Five cluster for a simpler model, and six clusters for a finer segmentation.

Perform K-Means clustering initializing and executing the k-means clustering algorithm on the df_4_processed dataset, specifying 5 clusters based on the earlier analysis with the elbow method.

In [83]:
n_groups = 5
kmeans = KMeans(n_clusters=n_groups, random_state=42, n_init=10)
kmeans.fit(df_4_processed.values)
km_labels = kmeans.labels_

Assign cluster labels to df_4 DataFrame generated by the k-means clustering algorithm.

In [84]:
df_4["km_labels"] = km_labels

Display the first five rows of df_4.

In [85]:
df_4.head()

,federal_action_sum,recipient_country,recipient_state,state_of_performance,award_type,type_of_contract_pricing,naics,extent_competed,c8a_program,number_of_contracts,km_labels
0,45935.00,united states,ca,ca,purchase order,firm fixed price,337,not competed under sap,f,1,2
1,4463.00,united states,dc,dc,bpa call,firm fixed price,541,competed under sap,f,4,0
2,171001.76,united states,tx,tx,purchase order,firm fixed price,541,competed under sap,f,3,2
3,7515586.92,united states,in,in,delivery order,firm fixed price,311,full and open competition,f,11,1
4,1702950.88,united states,nh,co,delivery order,firm fixed price,334,full and open competition,f,3,1


Check the structure of df_4.

In [86]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86145 entries, 0 to 86144
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   federal_action_sum        86145 non-null  float64 
 1   recipient_country         86145 non-null  category
 2   recipient_state           86145 non-null  category
 3   state_of_performance      86145 non-null  category
 4   award_type                86145 non-null  category
 5   type_of_contract_pricing  86145 non-null  category
 6   naics                     86145 non-null  category
 7   extent_competed           86145 non-null  category
 8   c8a_program               86145 non-null  category
 9   number_of_contracts       86145 non-null  int64   
 10  km_labels                 86145 non-null  int32   
dtypes: category(8), float64(1), int32(1), int64(1)
memory usage: 2.3 MB


Create a pivot table for cluster analysis from the df_4 DataFrame, aggregating data based on the k-means cluster labels. The pivot table index is km_labels which are the labels assigned from the k-means clustering, used as the primary grouping variable. The values are aggregations performed on federal_action_sum and number_of_contracts, providing insights into the financial activity and contract volume within each cluster.

In [87]:
df_4_pivot = pd.pivot_table(
    df_4,
    values=[
        "federal_action_sum",
        "number_of_contracts",
        ],
    index=["km_labels"],
    aggfunc={
        "federal_action_sum": ["count", "sum", "mean"],
        "number_of_contracts": ["sum", "mean"],
        }
        )

Display the pivot table.

In [88]:
df_4_pivot

federal_action_sum                             number_of_contracts  \
                       count          mean           sum                mean   
km_labels                                                                      
0                      12085  9.079101e+05  1.097209e+10           14.872900   
1                      30247  1.842946e+07  5.574358e+11          108.519027   
2                      43793  5.076448e+05  2.223129e+10           15.221063   
3                         13  1.058945e+10  1.376628e+11         3401.615385   
4                          7  1.065282e+09  7.456975e+09       258521.285714   

                    
               sum  
km_labels           
0           179739  
1          3282375  
2           666576  
3            44221  
4          1809649

Format pivot table column names to simplify and standardize the names in the df_4_pivot DataFrame.

In [89]:
df_4_pivot.columns = ['_'.join(col).strip() for col in df_4_pivot.columns.values]

Simplify and standardize the column names in the df_4_pivot DataFrame by concatenating the multi-level column indices into single-level names.

In [90]:
df_4_pivot = df_4_pivot.rename(
    columns={
        'federal_action_sum_count': 'number_of_companies',
        'federal_action_sum_mean': 'federal_action_mean',
        'federal_action_sum_sum': 'federal_action_sum',
        'number_of_contracts_sum': 'number_of_contracts',
        }
        )

Display first five rows of the pivot table.

In [91]:
df_4_pivot.head()

,number_of_companies,federal_action_mean,federal_action_sum,number_of_contracts_mean,number_of_contracts
km_labels,,,,,
0,12085,9.079101e+05,1.097209e+10,14.872900,179739
1,30247,1.842946e+07,5.574358e+11,108.519027,3282375
2,43793,5.076448e+05,2.223129e+10,15.221063,666576
3,13,1.058945e+10,1.376628e+11,3401.615385,44221
4,7,1.065282e+09,7.456975e+09,258521.285714,1809649


Order the df_4_pivot DataFrame to prioritize clusters based on their total federal action obligations, from the highest to the lowest. This sorting allows to quickly identify which clusters are associated with the highest total financial engagements. This could be particularly useful for understanding the distribution of federal spending across different clusters, helping to pinpoint areas of major activity or investment.

In [92]:
df_4_pivot_sorted = df_4_pivot.sort_values(by='federal_action_sum', ascending=False)
df_4_pivot_sorted

,number_of_companies,federal_action_mean,federal_action_sum,number_of_contracts_mean,number_of_contracts
km_labels,,,,,
1,30247,1.842946e+07,5.574358e+11,108.519027,3282375
3,13,1.058945e+10,1.376628e+11,3401.615385,44221
2,43793,5.076448e+05,2.223129e+10,15.221063,666576
0,12085,9.079101e+05,1.097209e+10,14.872900,179739
4,7,1.065282e+09,7.456975e+09,258521.285714,1809649


Create new labels based on sorted order and map the new labels to a new column in the original DataFrame df_4.

In [93]:
new_labels = {old_label: chr(65 + i) for i, old_label in enumerate(df_4_pivot_sorted.index)}
df_4['cluster_label'] = df_4['km_labels'].map(new_labels)

In [94]:
df_4.head()

,federal_action_sum,recipient_country,recipient_state,state_of_performance,award_type,type_of_contract_pricing,naics,extent_competed,c8a_program,number_of_contracts,km_labels,cluster_label
0,45935.00,united states,ca,ca,purchase order,firm fixed price,337,not competed under sap,f,1,2,C
1,4463.00,united states,dc,dc,bpa call,firm fixed price,541,competed under sap,f,4,0,D
2,171001.76,united states,tx,tx,purchase order,firm fixed price,541,competed under sap,f,3,2,C
3,7515586.92,united states,in,in,delivery order,firm fixed price,311,full and open competition,f,11,1,A
4,1702950.88,united states,nh,co,delivery order,firm fixed price,334,full and open competition,f,3,1,A


In [95]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86145 entries, 0 to 86144
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   federal_action_sum        86145 non-null  float64 
 1   recipient_country         86145 non-null  category
 2   recipient_state           86145 non-null  category
 3   state_of_performance      86145 non-null  category
 4   award_type                86145 non-null  category
 5   type_of_contract_pricing  86145 non-null  category
 6   naics                     86145 non-null  category
 7   extent_competed           86145 non-null  category
 8   c8a_program               86145 non-null  category
 9   number_of_contracts       86145 non-null  int64   
 10  km_labels                 86145 non-null  int32   
 11  cluster_label             86145 non-null  object  
dtypes: category(8), float64(1), int32(1), int64(1), object(1)
memory usage: 3.0+ MB


Set these new labels as the index in the pivot table (optionally).

In [96]:
df_4_pivot_sorted['new_labels'] = df_4_pivot_sorted.index.map(new_labels)
df_4_pivot_sorted.set_index('new_labels', inplace=True)
df_4_pivot_sorted

,number_of_companies,federal_action_mean,federal_action_sum,number_of_contracts_mean,number_of_contracts
new_labels,,,,,
A,30247,1.842946e+07,5.574358e+11,108.519027,3282375
B,13,1.058945e+10,1.376628e+11,3401.615385,44221
C,43793,5.076448e+05,2.223129e+10,15.221063,666576
D,12085,9.079101e+05,1.097209e+10,14.872900,179739
E,7,1.065282e+09,7.456975e+09,258521.285714,1809649


Format numeric columns to show two decimal places for readability.

In [97]:
numeric_cols = df_4_pivot_sorted.select_dtypes(include=['float64', 'int64']).columns
format_dict = {col: "{:.2f}" for col in numeric_cols}
df_4_pivot_sorted = df_4_pivot_sorted.style.format(format_dict)
df_4_pivot_sorted

,number_of_companies,federal_action_mean,federal_action_sum,number_of_contracts_mean,number_of_contracts
new_labels,,,,,
A,30247.00,18429457.65,557435805565.23,108.52,3282375.00
B,13.00,10589448318.86,137662828145.22,3401.62,44221.00
C,43793.00,507644.79,22231288471.52,15.22,666576.00
D,12085.00,907910.12,10972093739.95,14.87,179739.00
E,7.00,1065282181.71,7456975271.95,258521.29,1809649.00


### 4.1. Clustering conclusions

#### Analysis of Federal Contract Clusters by Labels

The pivot table organizes clusters labeled from A to E, representing various groups of companies based on their activity in federal contracts. These labels help elucidate the relative financial and operational magnitude of each cluster:

#### Cluster A
- **Number of Companies**: 30,247
- **Average Federal Action**: $18,429,457,65
- **Total Federal Action**: $557,435,805,565.23
- **Average Number of Contracts**: 108.52
- **Total Number of Contracts**: 3,282,375

**Interpretation**: Cluster A represents the largest cluster both in terms of total federal action and the number of contracts. It is likely composed of high-value, high-volume contractors, making it a crucial segment in federal contracting.

#### Cluster B
- **Number of Companies**: 13
- **Average Federal Action**: $10,589,448,318.86
- **Total Federal Action**: $137,662,828,145.22
- **Average Number of Contracts**: 3,401.62
- **Total Number of Contracts**: 44,221

**Interpretation**: Despite its small size, Cluster B involves a few companies handling extremely large contracts. This cluster likely includes major defense or technology contractors given the high average contract value.

#### Cluster C
- **Number of Companies**: 43,793
- **Average Federal Action**: $507,644.79
- **Total Federal Action**: $22,312,884,471.52
- **Average Number of Contracts**: 15.22
- **Total Number of Contracts**: 666,576

**Interpretation**: Cluster C contains the most companies, which are engaged in a significant number of contracts but at relatively lower values per contract, indicating a group of smaller or specialized providers.

#### Cluster D
- **Number of Companies**: 12,085
- **Average Federal Action**: $907,910.12
- **Total Federal Action**: $10,972,093,739.95
- **Average Number of Contracts**: 14.87
- **Total Number of Contracts**: 179,739

**Interpretation**: Cluster D showcases a mid-range cluster with moderate contract values and quantities, representing a vital middle-market segment in the federal contracting ecosystem.

#### Cluster E
- **Number of Companies**: 7
- **Average Federal Action**: $1,065,282,181.71
- **Total Federal Action**: $7,456,975,271.95
- **Average Number of Contracts**: 258,521.29
- **Total Number of Contracts**: 1,809,649

**Interpretation**: Cluster E, though small in company count, handles a significant number of contracts and a substantial total federal action, highlighting high-efficiency entities possibly involved in sectors like logistics or manufacturing support.

## 5. CSV file creation

Export the DataFrame df_4 which contains all labeled data, to a csv file to use in the following notebook (4_classification_models.ipynb).

In [98]:
df_4.to_csv("df_4.csv", index=False)